<a href="https://colab.research.google.com/github/wooohun/BERT-Summarizer/blob/main/BART_Abstractive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [4]:
# install transformers from huggingface
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.8 MB/s eta 0:00:00


In [2]:
# install kaggle
!pip install -q kaggle
!mkdir ~/.kaggle

# move kaggle api token to kaggle folder
!cp -v kaggle.json ~/.kaggle

'kaggle.json' -> '/root/.kaggle/kaggle.json'


In [3]:
# download dataset
# !chmod 600 /root/.kaggle/kaggl
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail
!unzip newspaper-text-summarization-cnn-dailymail

 97% 489M/503M [00:10<00:00, 65.0MB/s]
100% 503M/503M [00:10<00:00, 49.5MB/s]
Archive:  newspaper-text-summarization-cnn-dailymail.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [5]:
# grab datasets
test = pd.read_csv("cnn_dailymail/test.csv")
train = pd.read_csv("cnn_dailymail/train.csv")
validate = pd.read_csv("cnn_dailymail/validation.csv")

In [6]:
from transformers import BartTokenizer, BartModel

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model  = BartModel.from_pretrained('facebook/bart-large')

In [ ]:
# tokenize articles, highlights for test, train datasets

test['article_tokens'] = test['article'].apply(tokenizer.tokenize)
train['article_tokens'] = train['article'].apply(tokenizer.tokenize)

test['summary_tokens'] = test['highlights'].apply(tokenizer.tokenize)
train['summary_tokens'] = train['highlights'].apply(tokenizer.tokenize)